In [1]:
from docx import *
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [2]:
# read file
# document = Document("[2]Dictionary IND-ENG-783-805.docx")
# document = Document("Preprocessing/208-230.docx")
# document = Document("Converted/pdf2go-execution.docx")
document = Document("RPL/B/7. Fixed/[7]Dictionary IND-ENG-1-69-execution.docx")

In [3]:
# for paragraph in document.paragraphs:
    
#     len(paragraph.runs)
    
# #     for run in paragraph.runs :
# #         print(run.text)
    
    
#     # first line indent (segitiga atas di ruler)
#     first = paragraph.paragraph_format.first_line_indent
    
#     # left indent (segitiga bawah di ruler)
#     left = paragraph.paragraph_format.left_indent
    
#     print(first,left,paragraph.text+"\n")

In [4]:
# Target yang di crawling
# [lema, sublema, gabungan(-,~), similar(panah,"and"), keterangan, akronim([]), terjemahan]

# CARA NGEDAPETIN (bantu isi cara ngedapetin atau ciri khas atau langkah2 buat ngedapetin datanya)
# [v] == sudah dibuat codingannya

# [v] LEMA : indent pertama,bold (simpan di tempLema)
# [v] LEMA (jika sebaris dg lema lainnya) : indent pertama, jika kondisi tempLema isi, jika tidak bisa distemming dan huruf awal kata sama seperti awal kata tempLema
# [v] LEMA (jika sebaris dg sublema) : indent kedua, jika kondisi tempSublema harus isi, jika tidak bisa distemming dan huruf awal kata sama seperti awal kata tempSublema

# [v] SUBLEMA : indent kedua,deteksi bold (simpan di tempSublema)
# [v] SUBLEMA (jika sebaris dg lema): indent pertama, jika kondisi tempLema isi, jika bisa distemming dan huruf awal kata tidak sama seperti awal kata tempLema
# [v] SUBLEMA (jika sebaris dg sublema lainnya) : indent kedua, jika kondisi tempSublema harus isi, jika bisa distemming dan huruf awal kata tidak sama seperti awal kata tempSublema

# [v] GABUNGAN(-,~) : deteksi kata italic, ambil kata-katanya simpan ke list tempGabungan, cek di list ada - atau ~ tidak, kalau ada berarti itu kata gabungan dan replace - dan ~ dengan lema/sublema 

# [v] SIMILAR ("and") : indent pertama/kedua, deteksi bold, jika kata sebelumnya adalah and maka similar
# SIMILAR (panah) :

# [v] KETERANGAN :

# [v] AKRONIM :

# TERJEMAHAN :

In [5]:
# Perisiapan Word :
# - ubah panah jadi >
# - –/~ yang reguler dan bold dijadikan italic
# - ubah font, kemudian tanda yang tidak terbaca seperti di kata "major" (-^w) diganti menjadi <
# - tanda yang tidak terbaca seperti di kata "brother-in-law" (-) diganti menjadi ^w
# - tanda "- " diganti menjadi "<", contoh me- ngelabui (PERTIMBANGKAN LAGI)
# - ubah tanda "..." menjadi "_" (garis bawah)
# - s.t. = something
# - s.o.'s = someone's
# - s.o. = someone
# - q.v. = which see (refer to that word)
# - k.o. = kind(s) of
# - o.s. = oneself
# - e.o. = eachother
# - replace .^w jadi .^p
# - menghapus setiap header halaman dan garisnya



In [6]:
def cleanWord(word) :
    #cek spasi pada kata (jika terdiri dari 2 kata)
    if re.search("\s", word) :
        word = word.split()

#         word[:] = [w for w in word if not (w in roman_numerals or re.search("\d", w) or word == "/" or w == "." or w == "(")]
        
        #update 1
        word[:] = [w for w in word if not (w in roman_numerals or re.match("\d", w))]
    
        word = " ".join(word)
        
        if len(word) < 1 :
            word = None

    #jika terdiri dari 1 kata
    else :
        #update 3
#         if word in roman_numerals or re.search("\d", word) or word == "/" or word == "." or word == "(":
        
        if word in roman_numerals or re.search("\d", word) :
            word = None
    
    return(word)

def replaceData (x) :
    for idx in range(len(x)):
        x[idx] = re.sub("¤","fi",x[idx])
        x[idx] = re.sub("¶","fl",x[idx])
        x[idx] = re.sub("\s-\s","-",x[idx])
        x[idx] = re.sub("-\s","",x[idx])
        x[idx] = re.sub("\s{2}|\s{3}"," ",x[idx])
        x[idx] = re.sub("\s*$|\xad|\xad\s*","",x[idx])
        x[idx] = re.sub("^\)|^\)\s*","",x[idx])
        x[idx] = re.sub("\(\s\)|\(\s=\)|\(=\)|\(=","",x[idx])
        x[idx] = re.sub("^\w\)\s*|\s\w\)$","",x[idx])
        x[idx] = re.sub("^\s*|^\s|^\s\w","",x[idx])
        x[idx] = re.sub("\[and","",x[idx])
        x[idx] = re.sub("\[|\[\s","",x[idx])
        x[idx] = re.sub("\]","",x[idx])
        x[idx] = re.sub("\s-\s|-\s","-",x[idx])
        x[idx] = re.sub("\s<\s|\s<|<\s|<","",x[idx])
        x[idx] = re.sub("^\s","",x[idx])
        x[idx] = re.sub("\s\)",")",x[idx])
        x[idx] = re.sub("\(\s","(",x[idx])
        x[idx] = re.sub("^/.*/","",x[idx])
        x[idx] = re.sub("/\s","/",x[idx])
        x[idx] = re.sub("\snya","nya",x[idx])
        x[idx] = re.sub("\skan$","kan",x[idx])
        x[idx] = re.sub("\skan\s","kan ",x[idx])
#         x[idx] = re.sub("\(=","(",x[idx])
        
#         x[idx] = re.sub(";|;\s","",x[idx])
       
        x[idx] = re.sub("\.$|\s\.$|\.\s$|\.\s","",x[idx])
#         x[idx] = re.sub("_","...",x[idx])
        
#         x[idx] = re.sub("s\*t\*|s\*t","something",x[idx])
# #         x[idx] = re.sub("s\*o\*'s","someone's",x[idx])
#         x[idx] = re.sub("s\*o\*","someone",x[idx])
#         x[idx] = re.sub("q\*v\*","which see (refer to that word)",x[idx])
#         x[idx] = re.sub("k\*o\*","kind(s) of",x[idx])
#         x[idx] = re.sub("o\*s\*","oneself",x[idx])
#         x[idx] = re.sub("e\*o\*","eachother",x[idx])
        
#         x[idx] = re.sub("\*\*|\s\*\*\s","-",x[idx])
        
        
        x[idx] = re.sub("someone ’s","someone’s",x[idx])
        x[idx] = re.sub("ke pada","kepada",x[idx])
    
    return x

# fungsi untuk menyimpan hasil ke bentuk list
def cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan):
    x = [lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan]
    
    x = replaceData(x)
    
    #split panah
    if x[4] != "" :        
        x[4] = re.split(",|;",x[4])
        listPanah = []
        
        for idx in range(len(x[4])):
            if '(' in x[4][idx] :
                x[4][idx], y = re.sub(r"[/(].*?[/))=]","",x[4][idx]),re.sub(r"[()=]","",x[4][idx])
                x[4][idx], y = re.sub(r"\s$","",x[4][idx]),re.sub(r"\s$","",y)
                x[4][idx] = re.sub("\s{2}|\s{3}"," ",x[4][idx])
#                 x[4].append(x)
                listPanah.append(y)
        
        if len(listPanah) != 0 :
            for val in listPanah :
                x[4].append(val)
    
    #split terjemahan
    if x[9] != "" :
        x[9] = re.split(",|;",x[9])
        x[9] = replaceData(x[9])
        
        if "" in x[9] :
            x[9].remove("")
        
        for idx in range(len(x[9])):
            x[9][idx] = re.sub("_","...",x[9][idx])
            x[9][idx] = re.sub("e\*g\*","e.g.,",x[9][idx])
            x[9][idx] = re.sub("i\*e\*","i.e.,",x[9][idx])
            x[9][idx] = re.sub("\*etc",", etc",x[9][idx])
            x[9][idx] = re.sub("etc \.","etc.",x[9][idx])
            x[9][idx] = re.sub("\*|\s\*",",",x[9][idx])
            
        if len(x[9]) < 1 :
            x[9] = ""
    
    y = pd.DataFrame(data=[x],columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","perbandingan","keterangan","terjemahan"])
    
    return x,y

def cleanParenthesis(word) :
    if '(' in word :
        x, y = re.sub(r"[/(].*?[/))=]","",word),re.sub(r"[()=]","",word)
        x, y = re.sub(r"\s$","",x),re.sub(r"\s$","",y)
        word = x
        similar = y
        tempLemaSublema = word
        boolLemaSublema = True
    else :
        word = re.sub(r"[()]|\s$","",word)
        similar = ""
        tempLemaSublema = word
        boolLemaSublema = True
        
    return word,similar,tempLemaSublema,boolLemaSublema

def readItalicData(word,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan) :     
    if word == "opp" or word == "cp":
        word = ""

    #KETERANGAN
    # mengambil keterangan
    if index > 0 and re.search("\(", paragraph.runs[index-1].text) and re.search("–|~", word) == None:

        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)
            
        else :
            #update 2
            
#             try :
            if (index+1 < len(paragraph.runs) and paragraph.runs[index+1].italic) or (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text)) :
                tempGabungan.append(word)
            else :
                if terjemahan == "" and word not in roman_numerals and boolPanah != True:
                    terjemahan = word
                elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                    terjemahan = terjemahan+" "+word
#             except :
#                 if terjemahan == "" and word not in roman_numerals and boolPanah != True:
#                     terjemahan = word
#                 elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
#                     terjemahan = terjemahan+" "+word

        # update sementara
        try :
            if (re.search("\)", word) and re.search("–|~", word) == None) or (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text) and re.search("–|~", word) == None):
                keterangan = " ".join(tempKeterangan)
                tempKeterangan = []
        except :
            if re.search("\)", word) and re.search("–|~", word) == None:
                keterangan = " ".join(tempKeterangan)
                tempKeterangan = []
        
#         keterangan = "satu"

    # mengambil keterangan selanjutnya jika terdiri dari > 1 keterangan
    elif len(tempKeterangan) > 0 and word not in tempKeterangan:
        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)

            try :
                if (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text)) or re.search("\)", word) :
                    keterangan = " ".join(tempKeterangan)
                    tempKeterangan = []
            except :
                print("terjadi kesalahan")
                
        else :
            if terjemahan == "" and word not in roman_numerals and boolPanah != True:
                terjemahan = word
            elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                terjemahan = terjemahan+" "+word
#         keterangan = "dua"

    else :
        tempGabungan.append(word)
#         keterangan = "tiga"
    
    return keterangan,tempKeterangan,tempGabungan,terjemahan

In [8]:
lexicon = pd.DataFrame(columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","perbandingan","keterangan","terjemahan"])
roman_numerals = ["I","II","III","IV","V","VI","VII","VIII","IX","X"]
# list_and = ["and", " and", "and ", " and ", "[and", "[and "]
list_ket = ["A","abbr","Ac","acr","adj","AE","anat","app","Bal","Ban","Bat","BD","BE","BG","bio","bot","C","chem","cla","col","coq","cp","D","D/E","derog","E","elec","e.o.","epist","esp","euph","exclam","fin","Fr","G","geo","geol","Gr","gram","Hind","IBT","infr","init","insur","Irja","Isl","J","J/Jv","joc","Jp","Jv","K","k.o.","L","leg","ling","lit","M","Mad","Mal","math","Med","med","mil","Min","mod","mus","naut","NTB","NTT","ob","obj","O jv","onom","opp","o.s.","Pal","Pap","Pers","petro","phys","pl","pl obj","pl subj","poet","Port","Pr","pron","q.v.","RC","reg","rel","rev","S","sg","Sg","Skr","Skr neo","sl","s.o.","s.o.'s","s.t.","stat","subj","Sum","Tag","Tam","usu","voc","vulg","zod","neo"]
list_data = ["something", "someone", "oneself", "each other","something ", "someone ", "oneself ", "each other "]

# list_data = ["something", "someone's", "someone", "which see (refer to that word)", "kind(s) of", "oneself", "each other","something ", "someone's ", "someone ", "which see (refer to that word) ", "kind(s) of ", "oneself ", "each other "]

#variabel
lema = ""
sublema = ""
gabungan = ""
similar = ""
panah = ""
akronim = ""
antonim = ""
perbandingan = ""
keterangan = ""
terjemahan = ""

boolLemaSublema = False
boolPanah = False
boolSimilar = False
tempLema = ""
tempSublema = ""
# tempLemaSublema = ""
tempTerjemahan = ""
tempGabungan = []
tempKeterangan = []
tempAkronim = []
index = 0

#looping paragraf tiap document
for paragraph in document.paragraphs:
    
    boolLemaSublema = False
    
    index = 0
    
    #menyimpan nilai indentasi (first indent dan left indent)
    first = paragraph.paragraph_format.first_line_indent
    left = paragraph.paragraph_format.left_indent
    
    #looping setiap kata di setiap paragrafnya
    for run in paragraph.runs:
        
#         run.text = re.sub("- ","",run.text)
        
        
#         print("word = "+run.text)
#         print("boolSimilar = "+str(boolSimilar))
        
#         try :
#             print("prev = "+paragraph.runs[index-1].text)
#         except :
#             print("prev = -")
        
#         print("word = "+run.text)
#         print((index > 0 and re.search("\(=", paragraph.runs[index-1].text)) or (index > 1 and re.search("\(=", paragraph.runs[index-2].text)))

# # #         print("index = "+str(index))
        
#         print("========================")
#         print("length = "+str(len(paragraph.runs)))
        
        #mengatasi left indent == none, dengan mengubah nilai variabel left dengan tempLeft (nilai pada left sebelumnya)
        if left == None :
            left = 228600
#             left = tempLeft
        
        #deteksi kata bold
        if run.bold and run.italic == None  :
            if run.text.isspace() == False :
                #baca indent lema
#                 if (((first == None or first == 0 or first == 76200) and (left <= 76835)) or ((first == -152400 or first == -153035) and left >= 227965)) or (first == -635 and (left == 76200 or left == 75565)) :      

                if (((first == None or first == 0 or first == 76200 or first == -635) and (left <= 76835)) or ((first == -152400 or first == -153035) and (left >= 227965 or left <= 76835))) :      

                    tempLeft = left

                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                        
                        data = False
                        for item in x[0]:
                            if item != "" :
                                data = True
                                
                        if data == True :
                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                    #variabel 
                    lema = ""
                    sublema = ""
                    gabungan = ""
                    similar = ""
                    panah = ""
                    akronim = ""
                    antonim = ""
                    perbandingan = ""
                    keterangan = ""
                    terjemahan = ""

                    boolPanah = False
                    boolSimilar = False
                    tempGabungan = []
                    tempKeterangan = []
                    tempAkronim = []

                    word = cleanWord(run.text)

                    if word != None :
                        
#                         print("run.text = "+run.text)
#                         print(word)
                        
                        #LEMA
                        #jika lema kosong atau belum terdapat lema, lema = word
                        if boolLemaSublema == False :

                            results = cleanParenthesis(word)

                            lema = results[0]
                            similar = results[1]
#                             tempLemaSublema = results[2]
                            boolLemaSublema = [3]
                            
                            tempLema = lema

                        #jika lema tidak kosong atau variabel lema sudah terisi
                        else :
#                             print("prev = "+paragraph.runs[index-1].text)
#                             print("word = "+word)
                            
                            #SIMILAR (AND)
                            #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                            if (index > 0 and re.search("and", paragraph.runs[index-1].text)) or (index > 1 and re.search("and", paragraph.runs[index-2].text)) or (index > 0 and re.search(",", paragraph.runs[index-1].text)) or (index > 1 and re.search(",", paragraph.runs[index-2].text)) or (index > 0 and re.search("\(=", paragraph.runs[index-1].text)) or (index > 1 and re.search("\(=", paragraph.runs[index-2].text)) :
#                                 print(word)
                                similar = word
    
#                                 print("similar = "+similar)

                            #jika bukan similar
                            else :  
                                #SUBLEMA(JIKA SEBARIS/SEPARAGRAF DENGAN LEMA)  
                                #jika sublema kosong
                                if sublema == "" :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                       
                                        data = False
                                        for item in x[0]:
                                            if item != "" :
                                                data = True

                                        if data == True :
                                            print(x[0])
                                            lexicon  = lexicon.append([x[1]],ignore_index=True)
                                    
                                    lema = ""
                                    similar = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""
                                    boolSimilar = False

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                                #jika sublema tidak kosong, maka simpan sublema ke list terlebih dahulu
                                else :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                    
                                        data = False
                                        for item in x[0]:
                                            if item != "" :
                                                data = True

                                        if data == True :
                                            print(x[0])
                                            lexicon  = lexicon.append([x[1]],ignore_index=True)
                                    
                                    lema = ""
                                    similar = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""
                                    boolSimilar = False

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                #baca indent sublema
                else :
                    tempLeft = left 

                    #deteksi kata bold
                    if(run.bold) and run.text.isspace() == False :

                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                            
                            data = False
                            for item in x[0]:
                                if item != "" :
                                    data = True
                                
                            if data == True :
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)
                            
                        #variabel
                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""

                        boolPanah = False
                        boolSimilar = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

                        word = cleanWord(run.text)

                        if word != None :
                            
#                             print("prev = "+paragraph.runs[index-1].text)
#                             print("word = "+word)
                            
                            #SUBLEMA
                            #jika sublema kosong
                            if boolLemaSublema == False :

                                if (index > 0 and re.search("and", paragraph.runs[index-1].text)) or (index > 1 and re.search("and", paragraph.runs[index-2].text)) or (index > 0 and re.search(",", paragraph.runs[index-1].text)) or (index > 1 and re.search(",", paragraph.runs[index-2].text)) or (index > 0 and re.search("\(=", paragraph.runs[index-1].text)) or (index > 1 and re.search("\(=", paragraph.runs[index-2].text)):
                                    similar = word
                                else :
                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
    #                                 tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                            #jika sublema tidak kosong
                            else :
#                                 try :
#                                     print("prev = "+paragraph.runs[index-1].text)
#                                 except :
#                                     print("prev = -")
#                                 print(word)
                                
                                #SIMILAR (AND)
                                #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                                if (index > 0 and re.search("and", paragraph.runs[index-1].text)) or (index > 1 and re.search("and", paragraph.runs[index-2].text)) or (index > 0 and re.search(",", paragraph.runs[index-1].text)) or (index > 1 and re.search(",", paragraph.runs[index-2].text)) or (index > 0 and re.search("\(=", paragraph.runs[index-1].text)) or (index > 1 and re.search("\(=", paragraph.runs[index-2].text)):
                                    similar = word
                                    
#                                     print("similar = "+similar)

                                #SUBLEMA (JIKA SUBLEMA SEBARIS/SEPARAGRAF DENGAN SUBLEMA YANG LAIN)
                                #jika bukan, maka simpan sublema ke list terlebih dahulu
                                else :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                        
                                        data = False
                                        for item in x[0]:
                                            if item != "" :
                                                data = True

                                        if data == True :
                                            print(x[0])
                                            lexicon  = lexicon.append([x[1]],ignore_index=True)
                                    
                                    lema = ""
                                    sublema = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""
                                    boolSimilar = False

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

        #deteksi kata italic (kata gabungan, keterangan)
#             print("sublema = "+sublema)
#             print("similar = "+similar)
#             print("tempLemaSublema = "+tempLemaSublema)
            
        elif (run.italic and run.bold) or run.italic :
            if run.text.isspace() == False :
                
#                 print("====================")
#                 print("word = "+run.text)
#                 print("prev = "+paragraph.runs[index-1].text)
                
#                 try :
#                     print("prev-2 = "+paragraph.runs[index-2].text)
#                 except :
#                     print("prev-2 tdk ada")
                
# #                 print(index)
# #                 print(index > 0)
# #                 print(paragraph.runs[index-1].text in list_data)
#                 print("boolSimilar = "+str(boolSimilar))
#                 print((index > 0 and paragraph.runs[index-1].text in list_and) or (index > 1 and paragraph.runs[index-2].text in list_and))

                if (index > 0 and re.search("and", paragraph.runs[index-1].text)) or (index > 1 and re.search("and", paragraph.runs[index-2].text)) or (index > 0 and re.search("\(=", paragraph.runs[index-1].text)) or (index > 1 and re.search("\(=", paragraph.runs[index-2].text)) :
#                 if (index > 0 and re.search("and", paragraph.runs[index-1].text)) or (index > 1 and re.search("and", paragraph.runs[index-2].text)) :
                    boolSimilar = True
#                     print(run.text)
#                     print(boolSimilar)
    
                if (index > 0 and (paragraph.runs[index-1].text in list_data)) or (index > 1 and (paragraph.runs[index-2].text in list_data)) or ((index > 0 and re.search("_|;", paragraph.runs[index-1].text)) or (index > 1 and re.search("_|;", paragraph.runs[index-2].text))) :
#                 if (index > 0 and (paragraph.runs[index-1].text in list_data)) or (index > 1 and (paragraph.runs[index-2].text in list_data)) or ((index > 0 and re.search("_", paragraph.runs[index-1].text) and re.search("–|~", run.text)) or (index > 1 and re.search("_", paragraph.runs[index-2].text)) and re.search("–|~", run.text)) :
#                 if (index > 0 and (paragraph.runs[index-1].text in list_data)) or (index > 1 and (paragraph.runs[index-2].text in list_data)) :
                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                        data = False
                        for item in x[0]:
                            if item != "" :
                                data = True
                                
                        if data == True :
                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                    lema = ""
                    sublema = ""
                    gabungan = ""
                    similar = ""
                    panah = ""
                    akronim = ""
                    antonim = ""
                    perbandingan = ""
                    keterangan = ""
                    terjemahan = ""
                            
                    boolPanah = False
                    boolSimilar = False
                    tempGabungan = []
                    tempKeterangan = []
                    tempAkronim = []
                    
                    results = readItalicData(run.text,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan)

                    keterangan = results[0]
                    tempKeterangan = results[1]
                    tempGabungan = results[2] 
                    terjemahan = results[3]
                    
#                     print(results)
                    
                elif boolPanah == False and ((index > 0 and re.search(">", paragraph.runs[index-1].text)) or (index > 1 and re.search(">", paragraph.runs[index-2].text))) :
                    word = re.sub("\.","", run.text)
                    word = cleanWord(word)
                    
#                     print(word)

                    if word != None :
                        panah = word
                        boolPanah = True
                        word = ""

                else :
                    results = readItalicData(run.text,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan)

                    keterangan = results[0]
                    tempKeterangan = results[1]
                    tempGabungan = results[2] 
                    terjemahan = results[3]

#                     print(results)
#                     print("disini")
#                 print(results)
#                 print("========================")
                
        # deteksi kata reguler (akronim, terjemahan)
        else : 
            if run.text.isspace() == False :
                word = run.text

                if word != None :
                    #GABUNGAN
#                     if "–" in tempGabungan or "~" in tempGabungan:
#                         gabungan = " ".join(tempGabungan)
#                         gabungan = re.sub("–|~",tempLemaSublema,gabungan)
#                         gabungan = re.sub("  "," ", gabungan)

#                         results = cleanParenthesis(gabungan)

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []

#                     print(tempGabungan)

                    temp = " ".join(tempGabungan)
    
#                     if "–" in tempGabungan:
                    if re.search("–", temp) :
#                         gabungan = " ".join(tempGabungan)
                        gabungan = re.sub("–",tempLema,temp)
                        gabungan = re.sub("  "," ", gabungan)

                        results = cleanParenthesis(gabungan)
                        
                        if (boolSimilar != True) :
                            gabungan = results[0]
                            similar = results[1]
                        else :
                            gabungan = ""
                            similar = results[0]
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                            data = False
                            for item in x[0]:
                                if item != "" :
                                    data = True
                                
                            if data == True :
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""
                            
                        boolPanah = False
                        boolSimilar = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []
                    
#                     elif "~" in tempGabungan:
                    elif re.search("~", temp) :
#                         gabungan = " ".join(tempGabungan)
                        gabungan = re.sub("~",tempSublema,temp)
                        gabungan = re.sub("  "," ", gabungan)

                        results = cleanParenthesis(gabungan)
                        
                        if (boolSimilar != True) :
                            gabungan = results[0]
                            similar = results[1]
                        else :
                            gabungan = ""
                            similar = results[0]
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                            data = False
                            for item in x[0]:
                                if item != "" :
                                    data = True
                                
                            if data == True :
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""
                            
                        boolPanah = False
                        boolSimilar = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []

                    else :
                        if len(tempGabungan) != 0:
                            
#                             print ("length = "+str(len(tempGabungan)))
#                             print ("index = "+str(index))
#                             print(tempGabungan)
#                             print("tempLema = "+tempLema )
#                             print("tempSublema = "+tempSublema )
                            

#                             temp = " ".join(tempGabungan)
    
#                             print("temp = "+temp)
#                             print(tempLema.lower() in temp.lower())
#                             print(tempSublema.lower() in temp.lower())
                            
                            if boolPanah == True and panah != word and word not in roman_numerals :
#                                 temp = " ".join(tempGabungan)
                                panah = panah+" "+temp

                                temp = ""
                                tempGabungan = []
                            
#                             elif len(tempGabungan) > 1 and (tempLema.lower() in temp.lower() or tempLemaSublema.lower() in temp.lower()) :
                            elif len(tempGabungan) > 1 and (tempLema.lower() in temp.lower() or ( tempSublema != "" and tempSublema.lower() in temp.lower())) :
#                                 temp = " ".join(tempGabungan)
                                
#                                 print("dikene")
    
                                gabungan = gabungan+" "+temp
                                gabungan = re.sub(r'  ',' ', gabungan)

                                results = cleanParenthesis(gabungan)

                                if (boolSimilar != True) :
                                    gabungan = results[0]
                                    similar = results[1]
                                else :
                                    gabungan = ""
                                    similar = results[0]

                                boolSimilar = False
                                tempGabungan = []
                            
                            elif index > 1 and len(tempGabungan) <= 3:
#                                 print("disini")
#                                 temp = " ".join(tempGabungan)

                                if (boolSimilar != True) :
                                    terjemahan = terjemahan+" "+temp
                                else :
                                    similar = temp
                                
                                boolSimilar = False
                                temp = ""
                                tempGabungan = []
            
#                                 print("hahaha")

                            else :
#                                 temp = " ".join(tempGabungan)
                                gabungan = gabungan+" "+temp
                                gabungan = re.sub(r'  ',' ', gabungan)

                                results = cleanParenthesis(gabungan)

                                if (boolSimilar != True) :
                                    gabungan = results[0]
                                    similar = results[1]
                                else :
                                    gabungan = ""
                                    similar = results[0]

                                boolSimilar = False
                                tempGabungan = []

                #AKRONIM
                # mengambil akronim
                
#                 print("============")
#                 print("prev = "+paragraph.runs[index-1].text)
#                 print("word = "+word)
#                 if (index > 0 and re.search("\[", paragraph.runs[index-1].text)) and re.search("and", word):
                if (index > 0 and re.search("\[", paragraph.runs[index-1].text)):
                    print(tempAkronim)
                    tempAkronim.append(word)

                    if (index+1 < len(paragraph.runs) and re.search("\]", paragraph.runs[index+1].text)) or re.search("\]", paragraph.runs[index].text) :
                        akronim = " ".join(tempAkronim)
                        tempAkronim = []

                # mengambil akronim selanjutnya jika terdiri > 1 kata
                elif len(tempAkronim) > 0 and word not in tempAkronim :
                    tempAkronim.append(word)

#                     try :
                    if (index+1 < len(paragraph.runs) and re.search("\]", paragraph.runs[index+1].text)) or re.search("\]", paragraph.runs[index].text) :
                        akronim = " ".join(tempAkronim)
                        akronim = re.sub(r'  ',' ', akronim)
                        tempAkronim = []
#                     except :
#                         if re.search("\]", paragraph.runs[index].text) :
#                             akronim = " ".join(tempAkronim)
#                             akronim = re.sub(r'  ',' ', akronim)
#                             tempAkronim = []
                else :
                    #PANAH   
                    
#                     print("prev-1 = "+paragraph.runs[index-1].text)
                    
#                     if index > 1 :
#                         print("prev-2 = "+paragraph.runs[index-2].text)
                        
#                     print("word = "+word)
#                     print("terjemahan = "+terjemahan)
#                     print("index = "+str(index))
#                     print("length = "+str(len(paragraph.runs)))
                    
                    if boolPanah == True and word not in roman_numerals :
                        tempWord = re.sub("\.","", word)
                        tempWord = cleanWord(tempWord)
                     
#                         print("word 2= "+word)
#                         print(tempWord)

                        if tempWord != None :
                            panah = panah+" "+tempWord
                            word = ""
                            
#                         print("panah 2 = "+panah)
                    
                    if boolPanah == True and re.search("\.", word) :
#                         print("word 3= "+word)
                        
                        panah = re.sub("\.|\s\.","",panah)
                        boolPanah = False
                        word = ""
#                         print("panah 3 = "+panah)
                    
                    
#                     if word != "" and word != "." and (paragraph.runs[index-1].text == ">" or (index > 1 and paragraph.runs[index-2].text == ">")) :
                    if word != "" and word != "." and ((index > 0 and re.search(">", paragraph.runs[index-1].text)) or (index > 1 and re.search(">", paragraph.runs[index-2].text))) :

#                         print("word 1= "+word)

                        word = re.sub("\.","", word)
                        word = cleanWord(word)

                        if word != None :
                            panah = word
                            boolPanah = True
                            word = ""
                            
#                         print("panah = "+panah)
                        
#                         print("panah 1 = "+panah)
#                             
#                             try :
#                                 print(paragraph.runs[index+1].text)
#                                 print(panah)
#                             except :
#                                 print(paragraph.runs[index+0].text)
#                                 print(panah)

                    elif (index > 0 and paragraph.runs[index-1].italic == True) and ((index > 0 and paragraph.runs[index-1].text == "opp") or (index > 1 and paragraph.runs[index-2].text == "opp")):
                        re.sub("\s","",word)

                        antonim = word
                
                    elif (index > 0 and paragraph.runs[index-1].italic == True) and ((index > 0 and paragraph.runs[index-1].text == "cp") or (index > 1 and paragraph.runs[index-2].text == "cp")):
                        re.sub("\s","",word)

                        perbandingan = word

                    else :

                        #TERJEMAHAN
                        if re.search("\]\s", word) :
                            word = re.sub("\]\s","",word)

                        if re.search("and", word) and ((index > 0 and (paragraph.runs[index-1].bold) == True or paragraph.runs[index-1].italic) == True ) and ((index+1 < len(paragraph.runs) and (paragraph.runs[index+1].bold == True or paragraph.runs[index+1].italic == True)) or (index+2 < len(paragraph.runs) and (paragraph.runs[index+2].bold == True or paragraph.runs[index+2].italic == True))):
                            word = ""

                        if word == "[" or word == "]" or re.search(">", word) :
                            word = ""

#                         if terjemahan == "" and word not in roman_numerals and boolPanah != True :
                        if terjemahan == "" and boolPanah != True :
                            terjemahan = word
#                         elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                        elif terjemahan != "" and boolPanah != True:
                            terjemahan = terjemahan+" "+word

#                         print("prev = "+paragraph.runs[index-1].text)
#                         print("word = "+word)
#                         print("terjemahan = "+terjemahan)
#                         print("run.text = "+run.text)
#                         if (re.search("\.|\!|\?", run.text) and ( index < len(paragraph.runs)-1 and paragraph.runs[index+1] == " " )) or re.match("\w\)", run.text) :
                        if (re.search("\.$|\!|\?", run.text)) or re.match("\w\)", run.text) :
        
#                             print("hihi")
            
                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                
                                data = False
                                for item in x[0]:
                                    if item != "" :
                                        data = True
                                
                                if data == True :
                                    print(x[0])
                                    lexicon  = lexicon.append([x[1]],ignore_index=True)

                            lema = ""
                            sublema = ""
                            gabungan = ""
                            similar = ""
                            panah = ""
                            akronim = ""
                            antonim = ""
                            perbandingan = ""
                            keterangan = ""
                            terjemahan = ""
                            
                            boolPanah = False
                            boolSimilar = False
                            tempGabungan = []
                            tempKeterangan = []
                            tempAkronim = []
                            
#                             boolPanah = False
        index += 1                 

['a', '', '', '', '', '', '', '', '', '']
['', '', '', 'A', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['the first letter of the Latin alphabet used for writing Indonesian']]
['a', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', 'abbr', ['are (a unit of square measure in the metric system', 'equal to 100 square meters or about 119.6 square yards)']]
['a', '', '', '', '', '', '', '', 'abbr', ['ampere']]
['a', '', '', '', '', '', '', '', '', ['ahh (hesitation)']]
['A', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['(in music) the sixth tone/note in the ascending scale of C major']]
['A', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['car license plate for Banten']]
['à', '', '', '', '', '', '', '', '', '']
['', '', '', '@', '', '', '', '', '', ['at']]
['', '', 'à Rp 1 000 , -', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['at Rp 1, 000.00']]
[]
['A ’45', '', '', '', '', 'An

['', '', '', '', '', '', '', '', '', ['to break (a promise/agreement)', 'betray (a secret) through carelessness']]
['', '', 'mengabaikan janji', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['to break a promise']]
['', '', '', '', '', '', '', '', '', ['to overlook (a fact)', 'miss (a chance)', 'not comply with']]
['', '', 'Sayang sekali Anda abaikan peluang emas itu', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['It’s a shame you missed that golden opportunity']]
['', '', '', '', '', '', '', '', '', ['to disappoint (someone’s expectations)']]
['', '', '', '', '', '', '', '', '', ['to let things drift/take their natural course', 'leave unfinished (of work)']]
['', '', 'mengabaikan segala sesuatu', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['to let things drift along/take their natural course']]
['', '', 'tidak mengabaikan usahanya', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['not slacken one’s efforts

['', '', '', '', '', '', '', '', '', ['not to have the slightest idea (about something)']]
['abd', '', '', '', ['ABDI'], '', '', '', '', '']
['abdas', '', '', '', '', '', '', '', 'Pers', ['(Muslim) ritual ablution before prayer']]
['', 'berabdas', '', '', '', '', '', '', '', ['to make one’s ritual ablution before prayer']]
['abdi', '', '', '', '', '', '', '', 'A', '']
['', '', '', '', '', '', '', '', '', ['(bought) slave']]
['', '', '', '', '', '', '', '', '', ['servant', '(domestic) helper']]
['', '', 'abdi dalam/dalem', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['(royal servant in the kraton']]
['', '', 'abdi dalem jajar', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', 'Jv', ['the lowest rank in the kraton administrative structure']]
['', '', 'abdi dalem putih', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', 'Jv', ['royal servant in charge of religious affairs']]
['', '', 'abdi-dalemisme', '', '', '', '', '', '', '']
['', '', '', '',

['abiz', '', '', '', ['ABIS'], '', '', '', 'BG', '']
['abjad', '', '', '', '', '', '', '', 'A', '']
['', '', '', '', '', '', '', '', '', ['alphabet (such as alif-ba-ta, etc.)']]
['', '', '', '', '', '', '', '', 'ob', ['the sequence of Arabic characters', 'each of which has a numerical value (used for counting years or for divination), i.e., alif = 1, ba = 2, kaf = 100, etc']]
['', '', 'menurut abjad', 'menurut aturan abjad', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['alphabetical', 'in alphabetical order']]
['', '', 'abjad Latin/Romawi', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['Latin alphabet']]
['', 'berabjad', '', '', '', '', '', '', '', ['alphabetical']]
['', '', '', '', '', '', '', '', '', ['with an alphabet']]
['', 'mengabjadkan', '', '', '', '', '', '', '', ['to alphabetize']]
['', 'pengabjadan', '', '', '', '', '', '', '', ['alphabetization']]
['abjadiah', '', '', '', ['MENURUT abjad', 'MENURUT aturan abjad'], '', '', '', '', ['alpha

['', '', '', '', '', '', '', '', '', ['never free from', 'never doesn’t (do something)']]
['', '', 'tidak pernah absén dalam', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['always be present/active in', 'never be away from']]
['', '', 'sudah tidak absén lagi', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['to be no longer in (our) service']]
['', '', '', '', '', '', '', '', '', ['to sign in']]
['', 'mengabsén', '', '', '', '', '', '', '', ['to call the roll', 'check attendance']]
['', '', 'mengabsén diri sendiri', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['to answer the roll call']]
['', 'absénan', 'daftar absénan', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['attendance list', 'roster']]
['', 'pengabsénan', '', '', '', '', '', '', '', ['taking attendance']]
['absénsi', '', '', '', '', '', '', '', 'D', ['roll call']]
['', '', 'daftar absénsi', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', 

...

['awing', '', '', '', '', '', '', '', 'Pr', ['thief']]
['awit', '', '', '', '', '', '', '', '', '']
['', '', '', '', ['AWÉT'], '', '', '', 'ob', '']
['awit', '', '', '', '', '', '', '', 'J/Jv', '']
['', 'awitan', '', '', '', '', '', '', '', ['onset (of a disease)']]
['awloh', '', '', '', '', '', '', '', 'J', ['Allah', 'God']]
['awu', '', '', '', ['KALAH awu'], '', '', '', 'Jv', ['family relationship depending on the relative ages of the preceding generation']]
['awul-awul', '', '', '', '', '', '', '', '', ['cotton waste']]
['awung-awung', '', '', '', '', '', '', '', '', ['kind(s) of snack made from sticky-rice flour']]
['awur', '', '', '', '', '', '', '', 'Jv', '']
['', 'mengawur', '', '', '', '', '', '', '', '']
['', '', '', 'ngawur', '', '', '', '', 'coq', '']
['', '', '', '', '', '', '', '', '', ['to scatter', 'sow']]
['', '', '', '', '', '', '', '', '', ['at random', 'aimlessly', 'thoughtlessly', 'do something carelessly']]
['', 'mengawuri', '', '', '', '', '', '', '', ['to scatter

['', '', 'ayam termakan rambut', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['very nervous']]
['', '', '', '', '', '', '', '', 'sl', ['girl', 'chick (a young prostitute)']]
['', '', 'ayam aduan', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['fighting cock']]
['', '', 'ayam alas', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['wood cock/hen', 'jungle fowl', 'Gallus gallus']]
['', '', '', '', '', '', '', '', 'euph', ['asing pork']]
['', '', 'ayam babon', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['laying hen']]
['', '', 'ayam bakaran', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['fast-growing breed of chicken']]
['', '', 'ayam balik', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['chicken with inverted feathers']]
['', '', 'ayam bangkas', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['cock with white spots on its feathers']]
['', '', 'ayam be

['', '', 'ayam mutiara', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['helmeted guinea fowl', 'Numida meleagris']]
['', '', 'ayam nakar', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['day of sacrifice (on the pilgrimage)']]
['', '', 'ayam negeri', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['pedigreed chicken']]
['', '', '', '', '', '', 'AYAM', '', '', ['kampung']]
['', '', 'ayam O', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['chicken cooked with taoco', 'ginger', 'and other ingredients']]
['', '', 'ayam panggang', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['barbecued chicken']]
['', '', 'ayam papak', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['tailless rooster']]
['', '', 'ayam pedaging', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['broiler (chicken)']]
['', '', '', '', '', '', '', '', '', ['(in Surabaya) pimp']]
['', '', '', '', ''

['', '', '', '', '', '', '', '', '', ['until the end of his life']]
['Ayatollah', '', '', '', '', '', '', '', '', '']
['', '', '', 'Ayatullah', '', '', '', '', 'A', ['“Sign/Miracle of God”', 'a respected religious teacher (most common among Shi’ite Muslims)']]
['ayé', '', '', '', ['SAYA'], '', '', '', 'J', '']
['ayem', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', 'J/Jv', '']
['', '', '', '', '', '', '', '', '', ['patient', 'calm', 'quiet', 'contented', 'satisfied', 'pleased']]
['', '', 'dengan ayem', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['in peace']]
['', '', 'ayem tentrem', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['contented', 'quiet']]
['', '', '', '', '', '', '', '', 'Jv', ['untroubled', 'comfortable']]
['ayeng-ayengan', '', '', '', '', '', '', '', 'J/Jv', '']
['', '', '', '', '', '', '', '', '', ['to go back and forth']]
['', '', '', '', '', '', '', '', '', ['to roam around', 'go round and round in circles

['', '', '', '', '', '', '', '', '', ['one’s fingers are itching (to hit him)']]
['', 'terayun-ayun', '', '', '', '', '', '', '', ['to sway', 'stagger', 'reel', 'pitch', 'rock']]
['', '', 'Dahan-dahan itu terayun-ayun karena tiupan angin', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['The branches were swaying in the breeze']]
['', 'ayunan', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['rocking', 'swaying', 'oscillation']]
['', '', 'ayunan tangan', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['motion/movement of the hand', 'gesture']]
['', '', '', '', '', '', '', '', '', ['(children’s) swing (playground apparatus)', 'hammock']]
['', '', '', '', '', '', '', '', '', ['(rocking) cradle']]
['', '', '', '', '', '', '', '', '', ['sling (for throwing stones)']]
['', '', '', '', '', '', '', '', '', ['fluctuation']]
['', '', 'ayunan (rém', 'ayunan rém', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', [', etc']]
['',

In [ ]:
# lexicon.to_excel (r'results-1-69.xlsx', index = False, header=True)